# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

import sys
sys.path.append('../')

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 36 new papers today
          10 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) is None:
            doc.abstract = paper['abstract']
            
        doc.comment = get_markdown_badge(paper_id) + " _" + paper['comments'] + "_"
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/9 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2204.02410
extracting tarball to tmp_2204.02410... done.
Retrieving document from  https://arxiv.org/e-print/2204.02416
extracting tarball to tmp_2204.02416... done.
Retrieving document from  https://arxiv.org/e-print/2204.02501
extracting tarball to tmp_2204.02501... done.
Retrieving document from  https://arxiv.org/e-print/2204.02523


/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: LatexWarning: Found documentclass in tmp_2204.02501/mnras_template.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2204.02523... done.
Retrieving document from  https://arxiv.org/e-print/2204.02540


/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: LatexWarning: Found documentclass in tmp_2204.02523/aassymbols.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2204.02540... done.
Retrieving document from  https://arxiv.org/e-print/2204.02716
extracting tarball to tmp_2204.02716... done.
Retrieving document from  https://arxiv.org/e-print/2204.02719
extracting tarball to tmp_2204.02719... done.
Retrieving document from  https://arxiv.org/e-print/2204.02780
extracting tarball to tmp_2204.02780... done.
Retrieving document from  https://arxiv.org/e-print/2204.02796
extracting tarball to tmp_2204.02796... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [14]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.02410-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.02410) | **How Cosmic Rays Mediate the Evolution of the Interstellar Medium**  |
|| Christine M. Simpson, et al. -- incl., <mark>Rowan Smith</mark> |
|*Comments*| *27 pages, 21 Figures, submitted to MNRAS, comments welcome*|
|**Abstract**| We explore the impact of diffusive cosmic rays (CRs) on the evolution of the interstellar medium (ISM) under varying assumptions of supernova explosion environment. In practice, we systematically vary the relative fractions of supernovae (SN) occurring in star-forming high-density gas and those occurring in random locations decoupled from star-forming gas to account for SN from run-away stars or explosions in regions that have been cleared by prior SN, stellar winds, or radiation. We explore various mixed models by adjusting these fractions relative to each other. We find that in the simple system of a periodic stratified gas layer the ISM structure will evolve to one of two solutions: a "peak driving" state where warm gas is volume filling or a "thermal runaway" state where hot gas is volume filling. CR pressure and transport are important factors that strongly influence the solution state the ISM reaches and have the ability to flip the ISM between solutions. Observable signatures such as gamma ray emission and HI gas are explored. We find that gamma ray luminosity from pion decay is largely consistent with observations for a range of model parameters. The thickness of the HI gas layer may be too compact, however, this may be due to a large cold neutral fraction of midplane gas. The volume fraction of hot gas evolves to stable states in both solutions, but neither settles to a Milky Way-like configuration, suggesting that additional physics which is omitted here (e.g. a cosmological circum-galactic medium, radiation transport, or spectrally resolved and spatially varying CR transport) may be required. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.02416-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.02416) | **Unexplored outflows in nearby low luminosity AGNs: the case of NGC 1052**  |
|| S. Cazzoli, et al. -- incl., <mark>L. Hermosa Muñoz</mark>, <mark>L. Hernández-García</mark>, <mark>C. Ramos Almeida</mark> |
|*Comments*| *A&A accepted 04/04/2022, 31 pages, 12 figures and 3 appendixes*|
|**Abstract**| Outflows play a central role in galaxy evolution shaping the properties of galaxies. Understanding outflows and their effects in low luminosity AGNs, such as LINERs, is essential (e.g. they are a numerous AGN population in the local Universe). We obtained VLT/MUSE and GTC/MEGARA optical IFS-data for NGC1052, the prototypical LINER. The stars are distributed in a dynamically hot disc, with a centrally peaked velocity dispersion map and large observed velocity amplitudes. The ionised gas, probed by the primary component is detected up to $\sim$30arcsec ($\sim$3.3 kpc) mostly in the polar direction with blue and red velocities ($\mid$V$\mid$$<$250 km/s). The velocity dispersion map shows a notable enhancement ($\sigma$$>$90 km/s) crossing the galaxy along the major axis of rotation in the central 10arcsec. The secondary component has a bipolar morphology, velocity dispersion larger than 150 km/s and velocities up to 660 km/s. A third component is detected but not spatially resolved. The maps of the NaD absorption indicate optically thick neutral gas with a velocity field consistent with a slow rotating disc ($\Delta$V = 77$\pm$12 km/s) but the velocity dispersion map is off-centred without any counterpart in the flux map. We found evidence of an ionised gas outflow with mass of 1.6$\pm$0.6 $\times$ 10$^{5}$ Msun, and mass rate of 0.4$\pm$0.2 Msun/yr. The outflow is propagating in a cocoon of gas with enhanced turbulence and might be triggering the onset of kpc-scale buoyant bubbles (polar emission). Taking into account the energy and kinetic power of the outflow (1.3$\pm$0.9 $\times$ 10$^{53}$ erg and 8.8$\pm$3.5 $\times$ 10$^{40}$ erg/s, respectively) as well as its alignment with both the jet and the cocoon, and that the gas is collisionally ionised, we consider that the outflow is jet-powered, although some contribution from the AGN is possible. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.02501-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.02501) | **Pulsating hydrogen-deficient white dwarfs and pre-white dwarfs observed  with TESS -- IV. Discovery of two new GW Vir stars: TIC0403800675 and  TIC1989122424**  |
|| Murat Uzundag, et al. -- incl., <mark>Leandro G. Althaus</mark> |
|*Comments*| *8 Pages. arXiv admin note: text overlap with arXiv:2108.11093*|
|**Abstract**| We present two new GW Vir-type pulsating white dwarf stars, TIC\,0403800675 (WD\,J115727.68-280349.64) and TIC\,1989122424 (WD J211738.38-552801.18) discovered in the Transiting Exoplanet Survey Satellite (TESS) photometric data. For both stars, the TESS light curves reveal the presence of oscillations with periods in a narrow range between 400 and 410\,s, which are associated with typical gravity ($g$)-modes. Follow-up ground-based spectroscopy shows that both stars have similar effective temperature ($T_\mathrm{eff} = 110,000 \pm 10,000$\,K) and surface gravity ($\log g = 7.5 \pm 0.5$), but different He/C composition (mass fractions): He\,=\,0.75 and C\,=\,0.25 for TIC\,0403800675, and He\,=\,0.50 and C\,=\,0.50 for TIC\,1989122424. By performing a fit to their spectral energy distributions, we found for both stars radii and luminosities of $R=0.019\pm0.002\,R_\odot$ and $\log(L/L_\odot)=1.68^{+0.15}_{-0.24}$, respectively. By employing evolutionary tracks of PG~1159 stars, we find the masses of both stars to be $0.56\pm0.18 M_{\odot}$ from the $\log g$-$T_\mathrm{eff}$ diagram and $0.60^{+0.11}_{-0.09} M_{\odot}$ from the Hertzsprung Russell diagram. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.02540-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.02540) | **Refinement of the convex shape model and tumbling spin state of (99942)  Apophis using the 2020-2021 apparition data**  |
|| <mark>H.-J. Lee</mark>, et al. -- incl., <mark>S. Greenstreet</mark>, <mark>R. Lees</mark> |
|*Comments*| *14 pages, 5 figures; Accepted for publication on Astronomy & Astrophysics*|
|**Abstract**| Context. The close approach of the near-Earth asteroid (99942) Apophis to Earth in 2029 will provide a unique opportunity to examine how the physical properties of the asteroid could be changed due to the Earth's gravitational perturbation. As a result, the Republic of Korea is planning a rendezvous mission to Apophis. Aims. Our aim was to use photometric data from the apparitions in 2020-2021 to refine the shape model and spin state of Apophis. Methods. Using thirty-six 1 to 2-m class ground-based telescopes and the Transiting Exoplanet Survey Satellite, we performed a photometric observation campaign throughout the 2020-2021 apparition. The convex shape model and spin state were refined using the light-curve inversion method. Results. According to our best-fit model, Apophis is rotating in a short axis mode with rotation and precession periods of 264.178 hours and 27.38547 hours, respectively. The angular momentum vector orientation of Apophis was found as (275$^\circ$, -85$^\circ$) in the ecliptic coordinate system. The ratio of the dynamic moments of inertia of this asteroid was fitted to $I_a:I_b:I_c=0.64:0.97:1$, which corresponds to an elongated prolate ellipsoid. These findings regarding the spin state and shape model could be used to not only design the space mission scenario but also investigate the impact of the Earth's tidal force during close encounters. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.02716-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.02716) | **A Novel Cloud-Based Framework for Standardised Simulations in the Latin  American Giant Observatory (LAGO)**  |
|| <mark>Antonio Juan Rubio-Montero</mark>, et al. -- incl., <mark>Hernán Asorey</mark> |
|*Comments*| *10 pages, 3 figures, Invited Talk at the Winter Simulation Conference WSC2021, Phoenix, AZ, USA*|
|**Abstract**| LAGO, the Latin American Giant Observatory, is an extended cosmic ray observatory, consisting of a wide network of water Cherenkov detectors located in 10 countries. With different altitudes and geomagnetic rigidity cutoffs, their geographic distribution, combined with the new electronics for control, atmospheric sensing and data acquisition, allows the realisation of diverse astrophysics studies at a regional scale. It is an observatory designed, built and operated by the LAGO Collaboration, a non-centralised alliance of 30 institutions from 11 countries. While LAGO has access to different computational frameworks, it lacks standardised computational mechanisms to fully grasp its cooperative approach. The European Commission is fostering initiatives aligned to LAGO objectives, especially to enable Open Science and its long-term sustainability. This work introduces the adaptation of LAGO to this paradigm within the EOSC-Synergy project, focusing on the simulations of the expected astrophysical signatures at detectors deployed at the LAGO sites around the World. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.02719-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.02719) | **A study of the F-giant star $θ$ Scorpii A: a post-merger rapid  rotator?**  |
|| <mark>Fiona Lewis</mark>, et al. -- incl., <mark>Floor van Leeuwen</mark> |
|*Comments*| *12 pages, 10 figures, Accepted by MNRAS*|
|**Abstract**| We report high-precision observations of the linear polarization of the F1$\,$III star $\theta$ Scorpii. The polarization has a wavelength dependence of the form expected for a rapid rotator, but with an amplitude several times larger than seen in otherwise similar main-sequence stars. This confirms the expectation that lower-gravity stars should have stronger rotational-polarization signatures as a consequence of the density dependence of the ratio of scattering to absorption opacities. By modelling the polarization, together with additional observational constraints (incorporating a revised analysis of Hipparcos astrometry, which clarifies the system's binary status), we determine a set of precise stellar parameters, including a rotation rate $\omega\, (= \Omega/\Omega_{\rm c})\ge 0.94$, polar gravity $\log{g_p} = 2.091 ^{+0.042}_{-0.039}$ (dex cgs), mass $3.10 ^{+0.37}_{-0.32}$ solar masses, and luminosity $\log(L/Lsun) =3.149^{+0.041}_{-0.028}$. These values are incompatible with evolutionary models of single rotating stars, with the star rotating too rapidly for its evolutionary stage, and being undermassive for its luminosity. We conclude that $\theta$ Sco A is most probably the product of a binary merger. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.02780-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.02780) | **Eliminating Primary Beam Effect in Foreground Subtraction of Neutral  Hydrogen Intensity Mapping Survey with Deep Learning**  |
|| Shulei Ni, Yichao Li, Li-Yang Gao, <mark>Xin Zhang</mark> |
|*Comments*| *17 pages, 12 figures*|
|**Abstract**| In the neutral hydrogen (HI) intensity mapping (IM) survey, the foreground contamination on the cosmological signals is extremely severe, and the systematic effects caused by radio telescopes themselves further aggravate the difficulties in subtracting foreground. In this work, we investigate whether the deep learning method, concretely the 3D U-Net algorithm here, can play a crucial role in foreground subtraction when considering the systematic effect caused by the telescope primary beam. We consider two beam models, i.e., the Gaussian beam model as a simple case and the Cosine beam model as a sophisticated case. To make a comparison, and also a combination, with the U-Net method, we also employ the traditional principal component analysis (PCA) method in the foreground subtraction. We find that, in the case of the Gaussian beam, both the PCA and U-Net methods can effectively clean the foreground, but in the case of the Cosine beam, U-Net performs much better than PCA in cleaning the foreground. In order to show how well the PCA and U-Net methods can recover the HI signals, we also derive the angular power spectra, as well as the 2D power spectrum of HI after performing the foreground subtractions. It is found that, in the case of Gaussian beam, the concordance with the original HI map using U-Net is better than that using PCA by $27.4\%$, and in the case of Cosine beam, the concordance using U-Net is better than that using PCA by $144.7\%$. Therefore, the U-Net based foreground subtraction can efficiently eliminate the telescope primary beam effect and shed new light on recovering the HI power spectrum for future HI IM experiments. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.02796-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.02796) | **Molecules in the carbon-rich protoplanetary nebula CRL 2688**  |
|| Jian-Jie Qiu, <mark>Yong Zhang</mark>, <mark>Jiang-Shui Zhang</mark>, Jun-ichi Nakashima |
|*Comments*| *55 pages, 10 figures, Accepted for publication in ApJS*|
|**Abstract**| We present observations of the carbon-rich protoplanetary nebula (PPN) CRL 2688 made with the Institut de Radioastronomie Millimetrique (IRAM) 30 m telescope in the 3mm and 2mm bands. In total, 196 transition lines belonging to 38 molecular species and isotopologues are detected, among which, to our best knowledge, 153 transition lines and 13 species are the first report for this object. Additionally, in order to contribute to future research, we have collected observational data on the molecular lines of CRL 2688 from the literature and compiled them into a single unified catalog. We find that the molecular abundance of CRL 2688 cannot be explained by the standard model of a circumstellar envelope. The implications of metal-bearing molecules on circumstellar chemistry are discussed. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in sorted(res, key=lambda x:x[0])]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

14  publications in the last 7 days.
	 exports/2204.01758.md
	 exports/2204.02109.md
	 exports/2204.02017.md
	 exports/2204.01824.md
	 exports/2203.15811.md
	 exports/2203.16735.md
	 exports/2203.16504.md
	 exports/2203.15822.md
	 exports/2204.01245.md
	 exports/2204.00793.md
	 exports/2204.00342.md
	 exports/2203.16959.md
	 exports/2203.16856.md
	 exports/2203.16734.md


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers

In [13]:
raise NotImplementedError("Manual Stop")

NotImplementedError: Manual Stop

In [ ]:
from IPython.display import display, Markdown
from TexSoup import TexSoup
import re

def bracket_error(source: str):
    """ Find problematic portions of the document """
    
    print("len(source)", len(source))
    
    # Checking header
    begin_doc = next(re.finditer(r'\\begin\{document\}', doc.source)).span()[1]
    header = source[:begin_doc]
    text = header + r"\n\end{document}"

    try:
        # print("Header check... ", end='')
        TexSoup(text)
        display(Markdown(f"**[OK]** - Header"))
    except:
        raise RuntimeError("Error in the header")
        
    # Check the text per section until the end.
    # Do not stop and try them all.
    
    problematic_text = []
    
    sections = ([(0, begin_doc, 'until first section')] + 
                [(g.span()[0], g.span()[1], g.group()) for g in re.finditer(r'\\section\{.*\}', source)] +
                [(g.span()[0], g.span()[1], g.group()) for g in re.finditer(r'\\begin\{appendix\}', source)]
               )
    sections.append([len(source), len(source), 'end'])
    
    sections = sorted(sections, key=lambda x: x[0])
    
    prev_pos, prev_name = (0, 'header')
    parsed = []
    
    for span, span_end, name in sections:

        if span - prev_pos <= 0:
            continue
            

        text = source[prev_pos:span]
        if prev_pos > begin_doc:
            text = r"\n\begin{document}" + text + r"\n\end{document}"
        else:
            text = text + r"\n\end{document}"
        try:
            #print(f"{prev_pos}:{prev_name}-->{span}:{name} check... ", end='')
            parsed.append(TexSoup(text, tolerance=1))  # allow not ending env
            display(Markdown(f"**[OK]** - *{prev_pos}*:{prev_name} &rarr; *{span}*:{name}"))
            # print("ok")

            prev_pos = span
            prev_name = name
        except:
            # print(f"error between {prev_pos} and {span}")
            display(Markdown(f"**[ERR]** *{prev_pos}*:{prev_name} &rarr; *{span}*:{name}"))
            problematic_text.append((prev_pos, source[prev_pos:span]))
            prev_pos = span
            prev_name = name
            # raise
    return problematic_text, parsed


def check_environment(text, offset=0):
    """ Check environment """
    env = re.compile(r"\\begin\{(?P<env>.*)\}(.*)\\end\{(?P=env)\}", re.DOTALL)

    for match in env.finditer(text):
        beg, end = match.span()
        beg += offset
        end += offset
        envname = match.groups()[0]
        try:
            latex.TexSoup(match.group())
        except Exception as e:
            display(e)
            print(f"Error in {envname:s} between {beg} and {end}")
            return match.groups()[1], beg, end

In [ ]:
candidates[1]['identifier']

In [ ]:
import importlib
importlib.reload(latex)
which = "2204.02017"
paper_id = f'{which:s}'
folder = f'tmp_{paper_id:s}'
doc = latex.LatexDocument(folder)    

In [ ]:
doc

In [ ]:
#[check_environment(k) for k in bracket_error(doc.source)]
_, _, a = latex.get_content_per_section(doc.source, verbose=True)
if not a:
    print("no issues per section")
for ak in a:
    r = check_environment(ak[1], offset=ak[0])
    print(r[1], r[2])
    print(r[0])